In [21]:
import numpy as np
from scipy.optimize import minimize
from scipy.io import loadmat
from math import sqrt
import time
import csv
import pickle

def initializeWeights(n_in, n_out):
    """
    # initializeWeights return the random weights for Neural Network given the
    # number of node in the input layer and output layer

    # Input:
    # n_in: number of nodes of the input layer
    # n_out: number of nodes of the output layer
       
    # Output: 
    # W: matrix of random initial weights with size (n_out x (n_in + 1))"""

    epsilon = sqrt(6) / sqrt(n_in + n_out + 1)
    W = (np.random.rand(n_out, n_in + 1) * 2 * epsilon) - epsilon
    return W


def sigmoid(z):
    """# Notice that z can be a scalar, a vector or a matrix
    # return the sigmoid of input z"""

    return 1.0 / (1.0 + np.exp(-1.0 * z))


def preprocess():
    """ Input:
     Although this function doesn't have any input, you are required to load
     the MNIST data set from file 'mnist_all.mat'.

     Output:
     train_data: matrix of training set. Each row of train_data contains 
       feature vector of a image
     train_label: vector of label corresponding to each image in the training
       set
     validation_data: matrix of training set. Each row of validation_data 
       contains feature vector of a image
     validation_label: vector of label corresponding to each image in the 
       training set
     test_data: matrix of training set. Each row of test_data contains 
       feature vector of a image
     test_label: vector of label corresponding to each image in the testing
       set

     Some suggestions for preprocessing step:
     - feature selection"""

    mat = loadmat('mnist_all.mat')  # loads the MAT object as a Dictionary

    # Pick a reasonable size for validation data

    # ------------Initialize preprocess arrays----------------------#
    train_preprocess = np.zeros(shape=(50000, 784))
    validation_preprocess = np.zeros(shape=(10000, 784))
    test_preprocess = np.zeros(shape=(10000, 784))
    train_label_preprocess = np.zeros(shape=(50000,))
    validation_label_preprocess = np.zeros(shape=(10000,))
    test_label_preprocess = np.zeros(shape=(10000,))
    # ------------Initialize flag variables----------------------#
    train_len = 0
    validation_len = 0
    test_len = 0
    train_label_len = 0
    validation_label_len = 0
    # ------------Start to split the data set into 6 arrays-----------#
    for key in mat:
        # -----------when the set is training set--------------------#
        if "train" in key:
            label = key[-1]  # record the corresponding label
            tup = mat.get(key)
            sap = range(tup.shape[0])
            tup_perm = np.random.permutation(sap)
            tup_len = len(tup)  # get the length of current training set
            tag_len = tup_len - 1000  # defines the number of examples which will be added into the training set

            # ---------------------adding data to training set-------------------------#
            train_preprocess[train_len:train_len + tag_len] = tup[tup_perm[1000:], :]
            train_len += tag_len

            train_label_preprocess[train_label_len:train_label_len + tag_len] = label
            train_label_len += tag_len

            # ---------------------adding data to validation set-------------------------#
            validation_preprocess[validation_len:validation_len + 1000] = tup[tup_perm[0:1000], :]
            validation_len += 1000

            validation_label_preprocess[validation_label_len:validation_label_len + 1000] = label
            validation_label_len += 1000

            # ---------------------adding data to test set-------------------------#
        elif "test" in key:
            label = key[-1]
            tup = mat.get(key)
            sap = range(tup.shape[0])
            tup_perm = np.random.permutation(sap)
            tup_len = len(tup)
            test_label_preprocess[test_len:test_len + tup_len] = label
            test_preprocess[test_len:test_len + tup_len] = tup[tup_perm]
            test_len += tup_len
            # ---------------------Shuffle,double and normalize-------------------------#
    train_size = range(train_preprocess.shape[0])
    train_perm = np.random.permutation(train_size)
    train_data = train_preprocess[train_perm]
    train_data = np.double(train_data)
    train_data = train_data / 255.0
    train_label = train_label_preprocess[train_perm]

    validation_size = range(validation_preprocess.shape[0])
    vali_perm = np.random.permutation(validation_size)
    validation_data = validation_preprocess[vali_perm]
    validation_data = np.double(validation_data)
    validation_data = validation_data / 255.0
    validation_label = validation_label_preprocess[vali_perm]

    test_size = range(test_preprocess.shape[0])
    test_perm = np.random.permutation(test_size)
    test_data = test_preprocess[test_perm]
    test_data = np.double(test_data)
    test_data = test_data / 255.0
    test_label = test_label_preprocess[test_perm]

    # Feature selection
    # Your code here.
    
    #Select features using the training data and then apply the same selection to validation and test data.
    #I have slightly modified the assignment description (uploaded to ublearns) to reflect this change. 
    #The change is that we need you to submit the list of selected features also as part of the submission.
    #you need to record which features your network uses in the 'params.pickle' file.
   
    delCol = []
    bCol=train_data[0,0]    
    p = 28*28
    
    for i in range(p):
        train = all(z == bCol for z in train_data[:,i])
        if(train==True):
            delCol.append(i)  #delete the unnecessary columns
            
    print(delCol)
    train_data = np.delete(train_data, delCol, axis=1)
    validation_data = np.delete(validation_data, delCol, axis=1)
    test_data = np.delete(test_data, delCol, axis=1)
    allCol = np.arange(784)
    selected_features = np.delete(allCol,delCol)
    print('\n preprocess done')
   
    return train_data, train_label, validation_data, validation_label, test_data, test_label, selected_features


def nnObjFunction(params, *args):
    """% nnObjFunction computes the value of objective function (negative log 
    %   likelihood error function with regularization) given the parameters 
    %   of Neural Networks, thetraining data, their corresponding training 
    %   labels and lambda - regularization hyper-parameter.

    % Input:
    % params: vector of weights of 2 matrices w1 (weights of connections from
    %     input layer to hidden layer) and w2 (weights of connections from
    %     hidden layer to output layer) where all of the weights are contained
    %     in a single vector.
    % n_input: number of node in input layer (not include the bias node)
    % n_hidden: number of node in hidden layer (not include the bias node)
    % n_class: number of node in output layer (number of classes in
    %     classification problem
    % training_data: matrix of training data. Each row of this matrix
    %     represents the feature vector of a particular image
    % training_label: the vector of truth label of training images. Each entry
    %     in the vector represents the truth label of its corresponding image.
    % lambda: regularization hyper-parameter. This value is used for fixing the
    %     overfitting problem.
       
    % Output: 
    % obj_val: a scalar value representing value of error function
    % obj_grad: a SINGLE vector of gradient value of error function
    % NOTE: how to compute obj_grad
    % Use backpropagation algorithm to compute the gradient of error function
    % for each weights in weight matrices.

    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    % reshape 'params' vector into 2 matrices of weight w1 and w2
    % w1: matrix of weights of connections from input layer to hidden layers.
    %     w1(i, j) represents the weight of connection from unit j in input 
    %     layer to unit i in hidden layer.
    % w2: matrix of weights of connections from hidden layer to output layers.
    %     w2(i, j) represents the weight of connection from unit j in hidden 
    %     layer to unit i in output layer."""

    n_input, n_hidden, n_class, training_data, training_label, lambdaval = args

    w1 = params[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1)))
    w2 = params[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1)))
    obj_val = 0

    # Your code here
    #
    #
    #
    #
    #
    o = 0.0
    i=0
    trainSize= training_data.shape[0]
    training_data = np.append(training_data,np.ones([trainSize,1]),1)  #add column for bias
    training_data= training_data.T  #take the transpose

    hiddenOutput = sigmoid(np.dot(w1,training_data))  #multiply weights to training data using dot product
    
    hiddenOutputBias=hiddenOutput.T  #take the transpose
    hiddenOutputSize = hiddenOutputBias.shape[0]
    hiddenOutputBias = np.append(hiddenOutputBias,np.ones([hiddenOutputSize,1]),1)  #add column for bias  
    output = sigmoid(np.dot(w2,hiddenOutputBias.T))  #take the dot product with w2

    outputLabel = np.zeros((n_class,training_data.shape[1])) #initialize all output predicted labels to 0

    for i in range(len(training_label)):
        l=0
        l= int(training_label[i])
        outputLabel[l,i] = 1 # set class of true label
    

    #negative log-likelihood error calculation
    o += np.sum(outputLabel * np.log(output) + (1.0-outputLabel) * np.log(1.0-output))
    delOutput = output - outputLabel

    
    grad_w1 = 0.0
    grad_w2  = 0.0


    grad_w2 =  np.dot(delOutput.reshape((n_class,training_data.shape[1])), hiddenOutputBias)
    oDelSum = np.dot(delOutput.T,w2)


    oDelSum = oDelSum[0:oDelSum.shape[0], 0:oDelSum.shape[1]-1]
    delHidden = ((1.0-hiddenOutput) * hiddenOutput*oDelSum.T)
    grad_w1 =  np.dot(delHidden.reshape((n_hidden,training_data.shape[1])) , (training_data.T))


    o = ((-1)*o)/trainSize
    rand = np.sum(np.sum(w1**2)) + np.sum(np.sum(w2**2))
    o = o + ((lambdaval * rand) / (2.0*trainSize))
    
    grad_w1 = (grad_w1 + lambdaval * w1) / trainSize      
    grad_w2 = (grad_w2 + lambdaval * w2) / trainSize         
    obj_val = o
    obj_grad = np.array([])
    obj_grad = np.concatenate((grad_w1.flatten(), grad_w2.flatten()),0)


    # Make sure you reshape the gradient matrices to a 1D array. for instance if your gradient matrices are grad_w1 and grad_w2
    # you would use code similar to the one below to create a flat array
    # obj_grad = np.concatenate((grad_w1.flatten(), grad_w2.flatten()),0)
    

    return (obj_val, obj_grad)


def nnPredict(w1, w2, data):
    """% nnPredict predicts the label of data given the parameter w1, w2 of Neural
    % Network.

    % Input:
    % w1: matrix of weights of connections from input layer to hidden layers.
    %     w1(i, j) represents the weight of connection from unit i in input 
    %     layer to unit j in hidden layer.
    % w2: matrix of weights of connections from hidden layer to output layers.
    %     w2(i, j) represents the weight of connection from unit i in input 
    %     layer to unit j in hidden layer.
    % data: matrix of data. Each row of this matrix represents the feature 
    %       vector of a particular image
       
    % Output: 
    % label: a column vector of predicted labels"""

    labels = []
    for t in data:
        inputBias=np.hstack((t,np.ones(1)))
        h = np.dot(w1,inputBias)
        h = sigmoid(h)
        
        hBias=np.hstack((h,np.ones(1)))  #size (d+1)*1
        output = np.dot(w2,hBias)
        output=sigmoid(output) #k*1
        labels.append(np.argmax(output,axis=0))
        
    labels = np.array(labels)
    # Return a vector with labels   
    return labels


"""**************Neural Network Script Starts here********************************"""



train_data, train_label, validation_data, validation_label, test_data, test_label, selected_features = preprocess()

#  Train Neural Network

# set the number of nodes in input unit (not including bias unit)
n_input = train_data.shape[1]

# set the number of nodes in hidden unit (not including bias unit)
n_hidden = 100

# set the number of nodes in output unit
n_class = 10

# initialize the weights into some random matrices
initial_w1 = initializeWeights(n_input, n_hidden)
initial_w2 = initializeWeights(n_hidden, n_class)

# unroll 2 weight matrices into single column vector
initialWeights = np.concatenate((initial_w1.flatten(), initial_w2.flatten()), 0)

# set the regularization hyper-parameter


lambdaval = 10

#for lambdaval in range(0,65,5):
start_time = time.time()
args = (n_input, n_hidden, n_class, train_data, train_label, lambdaval)

# Train Neural Network using fmin_cg or minimize from scipy,optimize module. Check documentation for a working example

opts = {'maxiter': 50}  # Preferred value.

nn_params = minimize(nnObjFunction, initialWeights, jac=True, args=args, method='CG', options=opts)

# In Case you want to use fmin_cg, you may have to split the nnObjectFunction to two functions nnObjFunctionVal
# and nnObjGradient. Check documentation for this function before you proceed.
# nn_params, cost = fmin_cg(nnObjFunctionVal, initialWeights, nnObjGradient,args = args, maxiter = 50)


# Reshape nnParams from 1D vector into w1 and w2 matrices
w1 = nn_params.x[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1)))
w2 = nn_params.x[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1)))


#Creating the params.pickle file
with open('params.pickle', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([selected_features, w1,w2,n_hidden,lambdaval], f)


# Test the computed parameters

predicted_label = nnPredict(w1, w2, train_data)

# print current lambdaval
print('\n Current Lambda Value: ', lambdaval )

# find the accuracy on Training Dataset

print('\n Training set Accuracy:' + str(100 * np.mean((predicted_label == train_label).astype(float))) + '%')
tr_acc = 100 * np.mean((predicted_label == train_label).astype(float))


predicted_label = nnPredict(w1, w2, validation_data)

# find the accuracy on Validation Dataset

print('\n Validation set Accuracy:' + str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%')
v_acc = 100 * np.mean((predicted_label == validation_label).astype(float))

predicted_label = nnPredict(w1, w2, test_data)

# find the accuracy on Validation Dataset

print('\n Test set Accuracy:' + str(100 * np.mean((predicted_label == test_label).astype(float))) + '%')
te_acc = 100 * np.mean((predicted_label == test_label).astype(float))

finish_time = time.time() - start_time
print("\n Obtained time:%s" % (finish_time))

    #with open('graph2.csv', 'a') as myfile:
    #    writer = csv.writer(myfile, dialect='excel')
    #    bla = [lambdaval, tr_acc, v_acc,te_acc,finish_time]
    #    writer.writerow(bla)
    
    



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 52, 53, 54, 55, 56, 57, 59, 82, 83, 84, 85, 111, 112, 140, 141, 168, 476, 560, 644, 645, 671, 672, 673, 699, 700, 701, 727, 728, 729, 730, 754, 755, 756, 757, 758, 759, 780, 781, 782, 783]

 preprocess done

 Current Lambda Value:  10

 Training set Accuracy:95.58%

 Validation set Accuracy:94.97%

 Test set Accuracy:95.32%

 Obtained time:157.4793345928192
